In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/know_your_customer_use_case.ipynb">
      <img width="32px" src="[https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg](https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg)" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgrounding%2Fknow_your_customer_use_case.ipynb">
      <img width="32px" src="[https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN](https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN)" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/grounding/know_your_customer_use_case.ipynb">
      <img src="[https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg](https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg)" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/know_your_customer_use_case.ipynb">
      <img width="32px" src="[https://www.svgrepo.com/download/217753/github.svg](https://www.svgrepo.com/download/217753/github.svg)" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/know_your_customer_use_case.ipynb" target="_blank">
  <img width="20px" src="[https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg](https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg)" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/know_your_customer_use_case.ipynb" target="_blank">
  <img width="20px" src="[https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg](https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg)" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/know_your_customer_use_case.ipynb" target="_blank">
  <img width="20px" src="[https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg](https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg)" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/know_your_customer_use_case.ipynb" target="_blank">
  <img width="20px" src="[https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png](https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png)" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/know_your_customer_use_case.ipynb" target="_blank">
  <img width="20px" src="[https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg](https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg)" alt="Facebook logo">
</a>

| Author(s) |
| --- |
| [Lukas Geiger](https://github.com/ljogeiger) |

## Overview

This notebook demonstrates how to use the Gemini API (specifically the `gemini-2.5-flash-preview-05-20` model) to find and summarize negative news articles related to a specified entity. The entity can be a person, company, or ship. The notebook leverages Google Search as a tool for the Gemini API to ground its responses in real-world information.

You will learn how to:
* Configure the Gemini API client.
* Define a detailed system instruction to guide the model's behavior.
* Craft a prompt that incorporates an input entity.
* Use Google Search as a grounding tool for the model.
* Set generation parameters, including safety settings (note: this notebook explicitly turns them off for demonstration purposes, which may not be suitable for all use cases).
* Process the model's response to extract the generated text and grounding metadata (sources).

## Get started

### Install Google Gen AI SDK and other required packages
The following command installs the Google Generative AI SDK, which is necessary to interact with the Gemini API.

In [19]:
%pip install --upgrade --quiet google-genai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment. This allows the notebook to access Google Cloud services.

In [20]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Authenticate your notebook environment

In [21]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=0X5450qfBa9YrX2s62tepgL2IO6PFf&access_type=offline&code_challenge=7Aza6ij10F9iPpZ3FNXQQ9_P_YBgPDntqWEfCg6EodY&code_challenge_method=S256

^C


Command killed by keyboard interrupt



### Set Google Cloud project information and Initialize API Client

To get started using Vertex AI (which hosts the Gemini models used here), you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

This cell also initializes the `genai.Client` which will be used to interact with the Gemini API.

In [3]:
import os
from google import genai

PROJECT_ID = "sandbox-aiml"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    # Attempt to get project ID from environment variable if not set by user
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))
    if not PROJECT_ID or PROJECT_ID == 'None':
        raise ValueError("Please set your Google Cloud Project ID.")
print(f"Using Project ID: {PROJECT_ID}")

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1") # Default to us-central1 if not set
print(f"Using Location: {LOCATION}")

# Initialize the Google GenAI client
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

Using Project ID: sandbox-aiml
Using Location: us-central1


### Import libraries
Import necessary libraries, including `google.genai.types` for defining specific configurations for the API call, and `IPython.display` for better rendering of markdown in the notebook.

In [4]:
from google.genai import types
import json # Though not used in the final script, often useful for handling API responses
from IPython.display import Markdown, display

## Generating Negative News Reports with Gemini

### Load model
Specify the model ID to be used. We are using `gemini-2.5-flash-preview-05-20`, a fast and versatile model with reasoning capabilities.

In [5]:
MODEL_ID = "gemini-2.5-flash-preview-05-20"  # @param {type:"string"}

### Define Prompt Template and System Instructions

**System Instructions:** These provide high-level guidance to the model on its role, the desired output format, and steps to follow. We instruct the model to act as a professional report generator for negative news, to search thoroughly, cite dates, and handle cases where no negative news is found. A list of specific activities is provided to focus the search.

**Prompt Template:** This is the specific query sent to the model for each entity. It includes a placeholder `{entity}` which will be filled in with the actual entity name during execution.

In [11]:
prompt_template = """
Your task is to provide a comprehensive and professional report of negative news articles for a given input entity. The input entity can be a person, company, or ship.

Input Entity:
{input_entity}

Activities:
Money Laundering
Forgery
Bribery and Corruption
Human Trafficking

Follow these steps:

1.  If the input entity is a company, map it to its legal business name.
2.  Thoroughly search Google News for negative news articles related to the input entity and the specified activities across all time.
3.  Summarize and interpret the Google Search results for the given input entity and each activity. If there are no results for a given activity, skip it.
4.  For each activity with search results, create a headline that summarizes the event.
5.  Group the search results under the corresponding headline, including the date of each news article.
6.  For person names, strictly follow the entity names.
7.  If there are no negative news articles associated with the input entity, respond with: "There are no results found for {input_entity}."


Output Format:

Headline: [Summary of the event]
Date: [Date of the news article]
Summary: [Brief summary of the news article]

Example:

Headline: John Doe Accused of Money Laundering\n
Date: 2023-01-15\n
Summary: John Doe is accused of laundering money through offshore accounts, according to a report by the International Consortium of Investigative Journalists.

Ensure that the report is comprehensive, accurate, and professionally presented.
"""

system_instructions_text = """
You are a professional news analyst tasked with providing comprehensive reports on negative news articles related to a given entity. Your reports must be thorough, accurate, and professionally presented.
"""

### Helper Function to Get Sources
The `get_sources` function processes the `grounding_metadata` from the model's response. This metadata contains information about the web pages the model used to generate its answer (when using Google Search as a tool). The function extracts titles and URLs for these sources and formats them for display. This is crucial for verifying the information provided by the model.

In [7]:
def get_sources(response):
    """Return a formatted string of sources corresponding with response citations"""
    source_text = "\n\n**Sources:**\n"
    if not response.candidates or not response.candidates[0].grounding_metadata:
        return source_text + "No grounding metadata found.\n"

    metadata = response.candidates[0].grounding_metadata
    sources = {}
    source_titles = {}
    max_chunk_index = -1

    if not metadata.grounding_supports:
        return source_text + "No grounding supports found in metadata.\n"

    for support in metadata.grounding_supports:
        for chunk_index in support.grounding_chunk_indices:
            display_chunk_index = chunk_index + 1  # offset 0 list index
            if display_chunk_index > max_chunk_index:
                max_chunk_index = display_chunk_index
            if display_chunk_index not in source_titles and chunk_index < len(metadata.grounding_chunks):
                chunk = metadata.grounding_chunks[chunk_index]
                source_titles[display_chunk_index] = chunk.web.title
                sources[display_chunk_index] = chunk.web.uri # Corrected to use uri
            elif chunk_index >= len(metadata.grounding_chunks):
                 print(f"Warning: chunk_index {chunk_index} out of bounds for grounding_chunks (len: {len(metadata.grounding_chunks)}).")
    sorted_source_titles = dict(sorted(source_titles.items()))

    if sources:
        for i in sorted_source_titles:
            source_text += f"[[{i}] {sorted_source_titles[i]}]({sources[i]})\n"
    else:
        source_text += "No sources extracted from grounding metadata.\n"

    # Debugging information (optional, can be commented out)
    # print(f"Max Chunk Index: {max_chunk_index}")
    # print(f"Length of GroundingChunks: {len(metadata.grounding_chunks)}")
    return source_text

### Define Generation Function
The `generate_negative_news_report` function encapsulates the logic for calling the Gemini API. It takes an entity string and the system instructions text as input.
Key configurations:
* **`model`**: Uses the `MODEL_ID` defined earlier.
* **`contents`**: The user prompt, formatted with the specific entity.
* **`tools`**: Configured to use `types.GoogleSearch()`, enabling the model to perform Google searches to find relevant information.
* **`generate_content_config`**: 
    * `temperature=1`, `top_p=0.95`: These parameters control the randomness and creativity of the output. Higher temperature and top_p values lead to more diverse responses.
    * `max_output_tokens=8192`: Sets the maximum length of the generated response.
    * `response_modalities=["TEXT"]`: Specifies that we expect a text response.
    * **`safety_settings`**: **Important Note:** All harm categories (`HATE_SPEECH`, `DANGEROUS_CONTENT`, `SEXUALLY_EXPLICIT`, `HARASSMENT`) are set to `"OFF"`. This is done to ensure the model can retrieve and report on potentially sensitive topics related to negative news. However, in a production environment or for other use cases, you should carefully consider and configure appropriate safety settings based on your application's requirements and responsible AI practices.
    * `system_instruction`: The detailed instructions for the model's task.

The function calls `client.models.generate_content` and returns the API response.

In [14]:
def generate_kyc_report(entity_name: str, system_instructions: str):
    current_prompt = prompt_template.format(input_entity=entity_name)
    contents = [current_prompt]

    tools = [types.Tool(google_search=types.GoogleSearch()),]

    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        max_output_tokens=8192,
        response_modalities=["TEXT"], # Expect text modality output
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="BLOCK_NONE"), # Using new enums if applicable, else use "OFF"
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="BLOCK_NONE"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="BLOCK_NONE"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="BLOCK_NONE")
        ],
        tools=tools,
        system_instruction=types.Content(parts=[types.Part(text=system_instructions)]), # System instructions should be Content object
        thinking_config=types.ThinkingConfig(
            include_thoughts=True,
            ),
    )

    print(f"\n--- Generating report for: {entity_name} ---")
    response = client.models.generate_content(
        model=MODEL_ID, # Fully qualified model name
        contents=contents,
        config=generate_content_config, # Parameter name is generation_config
    )

    return response

### Define Entities and Run Analysis
Define a list of entities for which to generate reports. The code then iterates through this list, calls the `generate_negative_news_report` function for each entity, and displays the model's text response along with the extracted sources. Using `display(Markdown(...))` helps in rendering the output in a more readable format.

In [24]:
entities_to_check = ["Volkswagen", "Robert Burke"]  # Example entities, you can change or extend this list

for entity in entities_to_check:
    response = generate_kyc_report(entity, system_instructions_text)

    # Display the model's text response
    try:
        if response.candidates and len(response.candidates) > 0:
            candidate = response.candidates[0]
            if hasattr(candidate, 'content') and candidate.content:
                parts = candidate.content.parts
                if parts and len(parts) > 0:
                    # First display any thought parts
                    thought_parts = [part for part in parts if hasattr(part, 'thought') and part.thought]
                    if thought_parts:
                        display(Markdown("**Model Thoughts:**"))
                        for part in thought_parts:
                            if hasattr(part, 'text') and part.text:
                                display(Markdown(part.text))

                    # Then display the final response
                    response_parts = [part for part in parts if hasattr(part, 'thought') and not part.thought and hasattr(part, 'text') and part.text]
                    if response_parts:
                        display(Markdown("**Model Response:**"))
                        for part in response_parts:
                            display(Markdown(part.text))
                    else:
                        display(Markdown("**Model Response:**\nNo text content found in response parts."))
                else:
                    display(Markdown("**Model Response:**\nNo parts found in content."))
            else:
                display(Markdown("**Model Response:**\nNo content found in candidate."))
        else:
            display(Markdown("**Model Response:**\nNo candidates found in response."))
            if response.prompt_feedback:
                display(Markdown(f"Prompt Feedback: {response.prompt_feedback}"))
    except Exception as e:
        display(Markdown(f"**Error displaying response:** {str(e)}"))
        print("Full response object:", response)

    # Get and display grounding information (sources)
    sources_text = get_sources(response)
    display(Markdown(sources_text))

    print("------------------------------------------------------")


--- Generating report for: Volkswagen ---


**Model Thoughts:**

Examine the Search Plan

Okay, I've outlined the initial steps. First, confirm "Volkswagen" to "Volkswagen AG." Then, generate searches for each of the specified activities, adding "Volkswagen AG." Finally, I'll summarize the findings, group articles, and handle potential gaps in the results. This framework should deliver the requested report efficiently.




Evaluating Volkswagen's Legal Name

I've established "Volkswagen AG" as the search term. My initial sweep yielded results on money laundering, bribery/corruption, and human trafficking allegations. I found no direct "Volkswagen AG forgery" results, but I'm re-evaluating "dieselgate" and its potential implications for forgery of emissions data, using the articles.




**Model Response:**

...

## Evaluating Relevance: Bribery

Okay, the search results are clear. I'm seeing a strong concentration of recent articles directly linking Admiral Robert Burke to bribery and corruption. The other mentions of money laundering, forgery, and human trafficking seem to be about different people or are only tangentially related. The next step is to focus exclusively on the corruption angle.




**Model Response:**

Here is a comprehensive report of negative news articles for Robert Burke:

### Bribery and Corruption

Headline: Retired U.S. Navy Admiral Robert Burke Convicted in Bribery Scheme

Date: 2025-05-19, 2025-05-20
Summary: Retired U.S. Navy Admiral Robert Burke was convicted of bribery and other corruption charges. A federal jury found Burke guilty of conspiracy to commit bribery, bribery, performing acts affecting a personal financial interest, and concealing material facts from the United States. He was accused of steering a lucrative government contract to a company in exchange for a high-paying job with that same company after his retirement.
Burke ordered his staff to award a $355,000 contract to "Company A" (later identified as Next Jump) to train personnel under his command in Italy and Spain in December 2021. Prosecutors alleged that Burke, along with co-CEOs of the company, Yongchul "Charlie" Kim and Meghan Messenger, agreed that Burke would use his position to steer a contract in exchange for future employment. He began working at the company in October 2022 with a yearly starting salary of $500,000. This verdict makes Burke the most senior officer to be convicted in recent times for crimes committed while serving in the U.S. military. Sentencing for Burke is scheduled for August 22, 2025.



**Sources:**
[[1] justice.gov](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHpSOLo02y8nGTi4-89hpm7-zNP4hIoc4OJeV2hRzUMPdbkORBtsJBIu4aWTbqpl51Zev0q-6Bs1V4ap2xT0No615NjWMF23-jFDHSlEp5ln0lVv0LPWlBPc6V5ef5sG9seDc8KCFa73ZYAXGNxhfoU50SCkBztZMtHTDPP7vnnkMziCtsYipXGgcsONyALfx2C)
[[2] cbsnews.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEZbc-yNxKjDra6V-cTpR_A43b4cP5xIiCO7AQPSfcpm3PiX5FM5JbEDufYy0aeRzw2yWuuPq4b6O1-kETR5RuDn_dFMv_9K3Ou30UEx4T8Z-enPI8o1JFSJ9GaSqlWfsE8wzE1rrxbWrNoHavA1Vehlz6Hox9qsyLXrVh63Hw4AfBYV--yjw52hVuiCZSE-lNN5R5pWQ==)
[[3] foxnews.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG7DfF7GjKiAWAquWYFt7wMfBZiP9U8ZgG-bADvYFMCriujB4W69T1PFCDiLwUauRZ2OGb4gCBVcNLLvYsnFVsBwzGw_I6ICsXHGrYRyOEViolA4Qhqk6jp0aW8FGVDWFmYbVAWkXMFGXqvJulyGAtk8w5QlAuH_YLn94aAhOFgm8k1l54RPbcpz13STMFlsNRzKBLnnnH660j8pfGl)
[[4] indiatimes.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHIgPOXziZzqbMCh8as9SeouB1lzdDHJTHyHtVHE0Us83cMOtvxSYKbCzRDpF-CKhVaCXv3eauqHBnBiXgLVputdDs1nyFpDMNRqxptEeXusKIyhPtZA8JdX129TdQ4K7QPaOzX4XgEjyOqMXVe2n1hvRuEv0YTJ1tXdkU-YbVJcsNCXsuwdUeo5pFw_y4PGQSuCWUw92jjNept4kQn6HSoWvDxL9xN3VVhrlqy2H_JlVXGtC98gUtGeZ9cXl96h6bvDCPoSXPxXa8=)
[[5] military.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHJT4O8rpiifVIudXvI7Bp6lEYq6OJT-gSn3kPCqU83f90Ms7tZYimtTYpGdPNFFNEllcIrJuYviK3C2c7zMfDlo3KiM7HqLPURed0YtLZVkLbdu0mkvkXFyDzSz5LYwsyFw085OGh02newPkPTfloXs1i527s2NLqPXIbhadMNtptz3c1RRY8Skk4rBZEErZavcKkmA8lmEfaDKYhMgp-zQM98oN1A3M5M8IZRtp1hKcymLF8-ArKhGw==)
[[6] reddit.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGNRbHgS0USPv5yHTK4yF-g8jeXaW-8ISqPN8OEcNUrzzgnDq-6YwsIcwi4sIr878Zy0a7b2e0cUMHqygnWsjgI65Oac7zLyPRk6TdLmgjwGsQQuhMf4Ldem4qHWL0GgGP46_xfzNWrv_jwF_d3lFeTW16RX9_A_iZa45TwZxPTYxhG1pT-IlQO9fUKC_FLKRaQ5quhwm457ZkGAvajEWF6PNG1QRQ-)
[[7] yahoo.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXECx-Wvdg5IyuOhOS-M-PECvLe-2sDwzyMJU9UdZlHPPG6zRYtR2RBazrNn8wYtoRLIdlSOWXkualhiM43KNjNw_9opOlpttxCarOZ5shKvHbZyUwxR0gRWq2OZSJwN9g1NYOhthIzAqnfwIYDs2sLYTH_B6yd-0wU_DF0wgSMMpaw=)
[[8] justice.gov](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG8z5FZUpqsyxcjl2YSagCTbsz0l3eKwj3XkDX016RoYDNsJmGgUBrm4yPYR2FRHDS9-3XpLkVOzJeuLCFZZwj6CPPfVaUEZnDIkG-d1O7Eb_bP4f1kykDPXJkgALwBBh2gw-BRdjBdAB2MnZQY2GtNRzoulyoJ1YM3NKOspV775-0vd3fiXDJrRR004aayE3Bif8HV9l-mJ0NzkSiKlB1nLsh2D-vZYYLb8KANQJXMz2jQSBhC)
[[9] yahoo.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEXJtYntZ9mcEaDOvMWdyIqPZMubeSe-2saBIQ4oCjZSuQsxdP6HPkqwQtz3M9Lopq7TGIzr1v1XZ8nPOo7XtsX7F3VzZi4T3ZwFxwhuA6W_EE9umKUCMSJrPbGy26KCpOvtgnBdvLvs3QhqKk3b4CmqwH4o8Tb-wwbRLbK_Pp9jxU=)
[[10] dailyvoice.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFAiLfTxJ6udkobwCsZuM0sA_AiUW6u3P7_8uaQPCTH6wBMk-mXfbImLrwlzW0BHimzKFvkpO2Fj466C41HatpcEYRGnXvCX7MQGdXUHH_nPGVC1yy_sGwUoDjVr28XMcXIIn5j9gkafZc8WW0ikf5yLLnwPakMQKIBxGG14A==)


------------------------------------------------------


## Evaluation
Add section for evaluatin of response using Evaluation Framework

Import necessary libraries

In [25]:
from vertexai.evaluation import EvalTask, PointwiseMetric, PointwiseMetricPromptTemplate
import pandas as pd
from IPython.display import Markdown, display
import plotly.graph_objects as go

Define helper functions

In [26]:
def display_eval_result(eval_result, metrics=None):
    """Display the evaluation results."""
    summary_metrics, metrics_table = (
        eval_result.summary_metrics,
        eval_result.metrics_table,
    )

    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter([
            metric for metric in metrics_df.columns
            if any(selected_metric in metric for selected_metric in metrics)
        ])
        metrics_table = metrics_table.filter([
            metric for metric in metrics_table.columns
            if any(selected_metric in metric for selected_metric in metrics)
        ])

    # Display the summary metrics
    display(Markdown("### Summary Metrics"))
    display(metrics_df)
    # Display the metrics table
    display(Markdown("### Row-based Metrics"))
    display(metrics_table)

def plot_bar_plot(eval_results, metrics=None):
    """Create a bar plot of evaluation results."""
    fig = go.Figure()
    data = []

    for eval_result in eval_results:
        title, summary_metrics, _ = eval_result
        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        data.append(
            go.Bar(
                x=list(summary_metrics.keys()),
                y=list(summary_metrics.values()),
                name=title,
            ))

    fig = go.Figure(data=data)
    fig.update_layout(barmode="group")
    fig.show()

### Test Evaluation 1: Response Completeness and Accuracy

In [27]:
completeness_accuracy_template = PointwiseMetricPromptTemplate(
    criteria={
        "category_coverage": "The response should cover all relevant negative news categories (Money Laundering, Forgery, Bribery, Human Trafficking) if they exist for the entity.",
        "source_citation": "The response should properly cite sources for each claim."
    },
    rating_rubric={
        "5": "Response covers all relevant categories and properly cites sources.",
        "3": "Response covers most categories but may miss some citations.",
        "1": "Response is incomplete or contains inaccuracies."
    }
)

completeness_accuracy_metric = PointwiseMetric(
    metric="completeness_accuracy",
    metric_prompt_template=completeness_accuracy_template
)

The `input_variables` parameter is empty. Only the `response` column is used for computing this model-based metric.


In [28]:
# Create sample ground truth data
ground_truth_data = {
    "prompt": [
        """
Your task is to provide a comprehensive and professional report of negative news articles for a given input entity. The input entity can be a person, company, or ship.

Input Entity:
Volkswagen

Activities:
Money Laundering
Forgery
Bribery and Corruption
Human Trafficking

Follow these steps:

1.  If the input entity is a company, map it to its legal business name.
2.  Thoroughly search Google News for negative news articles related to the input entity and the specified activities across all time.
3.  Summarize and interpret the Google Search results for the given input entity and each activity. If there are no results for a given activity, skip it.
4.  For each activity with search results, create a headline that summarizes the event.
5.  Group the search results under the corresponding headline, including the date of each news article.
6.  For person names, strictly follow the entity names.
7.  If there are no negative news articles associated with the input entity, respond with: "There are no results found for Volkswagen."


Output Format:

Headline: [Summary of the event]
Date: [Date of the news article]
Summary: [Brief summary of the news article]

Example:

Headline: John Doe Accused of Money Laundering\n

Date: 2023-01-15\n

Summary: John Doe is accused of laundering money through offshore accounts, according to a report by the International Consortium of Investigative Journalists.

Ensure that the report is comprehensive, accurate, and professionally presented.
        """,
        """
Your task is to provide a comprehensive and professional report of negative news articles for a given input entity. The input entity can be a person, company, or ship.

Input Entity:
Robert Burke

Activities:
Money Laundering
Forgery
Bribery and Corruption
Human Trafficking

Follow these steps:

1.  If the input entity is a company, map it to its legal business name.
2.  Thoroughly search Google News for negative news articles related to the input entity and the specified activities across all time.
3.  Summarize and interpret the Google Search results for the given input entity and each activity. If there are no results for a given activity, skip it.
4.  For each activity with search results, create a headline that summarizes the event.
5.  Group the search results under the corresponding headline, including the date of each news article.
6.  For person names, strictly follow the entity names.
7.  If there are no negative news articles associated with the input entity, respond with: "There are no results found for Robert Burke."


Output Format:

Headline: [Summary of the event]
Date: [Date of the news article]
Summary: [Brief summary of the news article]

Example:

Headline: John Doe Accused of Money Laundering\n

Date: 2023-01-15\n

Summary: John Doe is accused of laundering money through offshore accounts, according to a report by the International Consortium of Investigative Journalists.

Ensure that the report is comprehensive, accurate, and professionally presented.
        """
    ],
    "reference": [
        """Here is a comprehensive report of negative news articles for Volkswagen, based on the provided activities. The legal business name used for the search was "Volkswagen AG".

Headline: Volkswagen Accused of Complicity in Human Trafficking and Forced Labor
Date: 2022-05-30 Summary: Volkswagen is accused of "slavery-like practices" and "human trafficking" in Brazil during the military dictatorship between 1974 and 1986. The Brazilian judiciary is examining complaints that allege the car manufacturer was complicit in "systematic human rights violations" on a cattle farm in the Amazon. Volkswagen has been summoned to appear before a labor court in Brasilia.

Date: Undated (likely 2024, given source content) Summary: Volkswagen's operations in China's Uyghur region are under increasing scrutiny due to concerns raised by human rights groups over forced labor involving Uyghur workers. The company's assembly plant in the region has faced criticism as evidence of Uyghur forced labor has surfaced, leading to restrictions on imports from the region by the U.S. and European markets.

Headline: Volkswagen Implicated in Money Laundering Through Board Member of Controlling Entity
Date: 2023-06-07 Summary: Siegfried Wolf, an Austrian businessman charged with money laundering in connection with the purchase of Eurofighter jets, sits on the supervisory board of Porsche SE, which is the holding company of Volkswagen Group. This indicates an indirect connection to Volkswagen through a board member of its controlling entity.

Headline: Volkswagen Charged with Fraud, Deception, and Emissions Test Manipulation ("Dieselgate")
Date: 2024-09-03 Summary: The criminal trial of former Volkswagen CEO Martin Winterkorn began over his role in the "dieselgate" emissions scandal. He is accused of fraud, market manipulation, and perjury for deliberately duping customers and shareholders by manipulating emissions tests.

Date: 2024-05-05 Summary: Defense attorneys are seeking an acquittal for former managers and engineers involved in the VW diesel fraud trial, accusing prosecutors of cynical and nonsensical arguments.

Date: 2024-04-12 Summary: A Volkswagen finance unit, Volkswagen Group of America Finance (VWGAF), was ordered to pay $48.8 million in an SEC fraud case related to the automaker's 2015 emissions violations. The SEC alleged that VWGAF made false and misleading statements in connection with its 2014 and 2015 corporate bond offerings, while senior executives knew that vehicles exceeded legal emissions limits.

Date: 2019-03-15 Summary: The U.S. Securities and Exchange Commission (SEC) charged Volkswagen AG, two of its subsidiaries, and its former CEO, Martin Winterkorn, for defrauding U.S. investors. They allegedly raised billions of dollars through bond markets while making deceptive claims about the environmental impact of the company's "clean diesel" fleet.

Date: 2018-05-03 Summary: Martin Winterkorn, the former chairman of the management board of Volkswagen AG, was charged with conspiracy and wire fraud in connection with Volkswagen's long-running scheme to cheat U.S. diesel vehicle emissions requirements.

Date: 2017-01-11 Summary: Volkswagen AG agreed to plead guilty to three criminal felony counts and pay a $2.8 billion criminal penalty, along with $1.5 billion in civil penalties, for its scheme to sell diesel vehicles in the U.S. using a "defeat device" to cheat emissions tests, and for lying and obstructing justice to further the scheme.

Date: 2015-12-10 Summary: Volkswagen admitted to installing "defeat devices" or software in diesel engines to detect when they were being tested, altering performance to improve results and cheat emissions tests. This "diesel dupe" affected approximately 11 million cars worldwide.

Sources: [1] scmp.com [2] business-humanrights.org [3] business-humanrights.org [4] humantraffickingsearch.org [5] freedomunited.org [6] amlintelligence.com [7] bbc.com [8] aljazeera.com [9] marketscreener.com [10] esgdive.com [11] complianceweek.com [12] sec.gov [13] justice.gov [14] justice.gov [15] bbc.com""",
        """Here is a comprehensive report of negative news articles for Robert Burke:

Bribery and Corruption

Headline: Retired Four-Star Admiral Robert Burke Convicted in Bribery Scheme Date: 2025-05-19 Summary: Retired four-star Admiral Robert Burke, 62, was found guilty of bribery, conspiracy to commit bribery, performing acts affecting a personal financial interest, and concealing material facts from the United States. He was convicted in connection with accepting future employment at a government vendor in exchange for awarding that company a government contract. Sentencing is scheduled for August 22, 2025.

Headline: Former Navy Admiral and Business Executives Arrested in Bribery Scheme Date: 2024-05-31 Summary: Robert Burke, a retired Navy Admiral, and two business executives, Yongchul "Charlie" Kim and Meghan Messenger, were arrested on charges related to an alleged bribery scheme. The indictment states that Burke accepted future employment at the executives' company in exchange for awarding them a U.S. government contract. The scheme involved Burke ordering his staff in December 2021 to award a $355,000 contract to Company A to train personnel under his command, and he began working at Company A in October 2022 with a $500,000 yearly salary and 100,000 stock options.

Headline: Retired US Navy Admiral Convicted on Corruption Charges Date: 2025-03-07 Summary: A jury convicted retired four-star US Admiral Robert Burke, who served as the US Navy's second-highest-ranking officer, on corruption charges. Burke was found guilty of steering a $355,000 contract to a company, then accepting a $500,000-a-year job with that company after his 2022 retirement from the Navy.

Sources: [1] justice.gov [2] justice.gov [3] regtechtimes.com [4] justice.gov [5] yahoo.com [6] bsky.app [7] yahoo.com [8] yahoo.com [9] yahoo.com [10] news9.com
"""
    ],
    "response": [
        """Here's a comprehensive report of negative news articles related to Volkswagen AG, focusing on the specified activities. The legal business name for Volkswagen is Volkswagen Aktiengesellschaft, often abbreviated as Volkswagen AG.

### Headline: Volkswagen Linked to Money Laundering Allegations via Controlling Shareholder

Date: 2023-06-07
Summary: An Austrian businessman, Siegfried Wolf, who was charged by an Austrian court on suspicion of money laundering related to the purchase of Eurofighter jets, sits on the supervisory board of Porsche SE, the holding company that controls Volkswagen Group.

### Headline: Volkswagen Accused of Forgery and Fraud in "Dieselgate" Emissions Scandal

Date: 2024-04-12
Summary: A Volkswagen finance unit was ordered to pay $48.8 million in a fraud case connected to the automaker's 2015 emissions violations. The SEC alleged that Volkswagen Group of America Finance made false and misleading statements in connection with bond offerings, knowing that over 500,000 vehicles in the U.S. exceeded legal emissions limits.

Date: 2019-03-15
Summary: The SEC charged Volkswagen AG, two of its subsidiaries, and its former CEO, Martin Winterkorn, for defrauding U.S. investors. Volkswagen allegedly raised billions of dollars through bond markets while making deceptive claims about the environmental impact of its "clean diesel" fleet, hiding its decade-long emissions scheme.

Date: 2018-05-03
Summary: Martin Winterkorn, the former chairman of the management board of Volkswagen AG, was charged with conspiracy and wire fraud in connection with the company's long-running scheme to cheat U.S. diesel vehicle emissions requirements. Volkswagen had pleaded guilty to criminal charges in 2017 for deceiving U.S. regulatory agencies by installing "defeat devices" designed to cheat emissions tests.

Date: 2017-01-11
Summary: Volkswagen AG agreed to plead guilty to three criminal felony counts and pay a $2.8 billion criminal penalty for a scheme to sell approximately 590,000 diesel vehicles in the U.S. using a defeat device to cheat on emissions tests, and for lying and obstructing justice. The company also agreed to pay $1.5 billion in separate civil resolutions for environmental, customs, and financial claims, including for customs fraud.

Date: 2015-12-10
Summary: Volkswagen admitted to cheating emissions tests in the US by installing "defeat devices" – software in diesel engines that could detect when they were being tested and change performance to improve results. This scandal, dubbed "dieselgate," affected about 11 million cars worldwide and led to significant financial penalties and a recall of millions of vehicles.

Date: Undated (Class period: 2010-11-19 to 2016-01-04)
Summary: A securities fraud class action was asserted against Volkswagen AG and its senior officers on behalf of purchasers of Volkswagen AG American Depositary Receipts (ADRs). Volkswagen was accused of systematically defrauding the public by deliberately cheating U.S. emissions tests and making its diesel vehicles appear cleaner than they were.

Date: 2025-05-05 (Published Date)
Summary: In an ongoing criminal trial related to the VW diesel scandal, defense attorneys for one of the former managers and engineers are seeking an acquittal, criticizing the prosecution's arguments.

### Headline: Volkswagen Implicated in Bribery and Corruption Scandal

Date: Undated
Summary: A bribery and corruption scandal at Volkswagen involved allegations that company cash was used to provide inappropriate benefits, such as pleasure trips, luxury hotel stays, and visits by prostitutes, for members of VW's works council. The scandal led to the dismissal of three top-ranking managers and the head of VW's general works council.

### Headline: Volkswagen Faces Allegations of Human Trafficking and Forced Labor

Date: 2024-05-27
Summary: Human Rights Watch reported that Volkswagen is applying inadequate oversight to the supply chains of its Chinese joint ventures, such as SAIC-VW, which primarily manufacture cars for sale in China. Allegations include potential links to Uyghur forced labor through sub-suppliers, despite Volkswagen stating that an audit found "no indications" of forced labor at its Xinjiang joint venture plant.

Date: 2023-03-30
Summary: Volkswagen failed to reach an agreement with the Public Ministry of Labour (MPT) in Brazil regarding compensations for alleged slave labor during the military dictatorship. Brazilian authorities accuse Volkswagen of systematic human rights violations in hundreds of cases between 1974 and 1986 on a cattle farm in the Amazon.

Date: 2022-05-30
Summary: Germany's Volkswagen Group faced accusations of "slavery" practices and human trafficking in Brazil during the country's military dictatorship in the 1970s and 1980s. The Brazilian judiciary is examining complaints alleging the car manufacturer used "slavery-like practices" and "human trafficking" and was complicit in "systematic human rights violations." Volkswagen was summoned to appear before a labor court in Brasilia.""",
        """Here's a comprehensive report of negative news articles related to Robert Burke:

### Headline: Retired Four-Star Admiral Robert Burke Convicted in Bribery Scheme

Date: 2025-05-19
Summary: Retired four-star Admiral Robert Burke, 62, was found guilty of bribery, conspiracy to commit bribery, performing acts affecting a personal financial interest, and concealing material facts from the United States. He was convicted in connection with accepting future employment at a government vendor in exchange for awarding that company a government contract. Sentencing is scheduled for August 22, 2025.

### Headline: Former Navy Admiral and Business Executives Arrested in Bribery Scheme

Date: 2024-05-31
Summary: Robert Burke, a retired Navy Admiral, and two business executives, Yongchul "Charlie" Kim and Meghan Messenger, were arrested on charges related to an alleged bribery scheme. The indictment states that Burke accepted future employment at the executives' company in exchange for awarding them a U.S. government contract. The scheme involved Burke ordering his staff in December 2021 to award a $355,000 contract to Company A to train personnel under his command, and he began working at Company A in October 2022 with a $500,000 yearly salary and 100,000 stock options.

### Headline: Retired US Navy Admiral Convicted on Corruption Charges

Date: 2025-03-07
Summary: A jury convicted retired four-star US Admiral Robert Burke, who served as the US Navy's second-highest-ranking officer, on corruption charges. Burke was found guilty of steering a $355,000 contract to a company, then accepting a $500,000-a-year job with that company after his 2022 retirement from the Navy."""
    ]
}

In [30]:
# Create evaluation dataset
eval_dataset = pd.DataFrame(ground_truth_data)

# Run evaluation
eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[completeness_accuracy_metric],
    experiment="kyc-completeness-accuracy"
)

eval_result = eval_task.evaluate()

# Display results
display_eval_result(eval_result)

Associating projects/305610648548/locations/us-central1/metadataStores/default/contexts/kyc-completeness-accuracy-5bcdffcc-c431-4c83-b216-b4e65a7e92df to Experiment: kyc-completeness-accuracy


Computing metrics with a total of 2 Vertex Gen AI Evaluation Service API requests.


100%|██████████| 2/2 [00:01<00:00,  1.14it/s]

All 2 metric requests are successfully computed.
Evaluation Took:1.840158999999403 seconds


### Summary Metrics

,row_count,completeness_accuracy/mean,completeness_accuracy/std
0,2.0,4.0,1.414214


### Row-based Metrics

,prompt,reference,response,completeness_accuracy/explanation,completeness_accuracy/score
0,\nYour task is to provide a comprehensive and ...,Here is a comprehensive report of negative new...,Here's a comprehensive report of negative news...,The response covers most categories and provid...,3.0
1,\nYour task is to provide a comprehensive and ...,Here is a comprehensive report of negative new...,Here's a comprehensive report of negative news...,The response covers all the negative news cate...,5.0


#### Test Evaluation 2: Response Structure and Professionalism

In [32]:
structure_professionalism_template = PointwiseMetricPromptTemplate(
    criteria={
        "formatting": "The response should follow the specified format with clear headlines, dates, and summaries.",
        "professional_tone": "The response should maintain a professional and objective tone throughout.",
        "clarity": "The information should be presented clearly and be easy to understand."
    },
    rating_rubric={
        "5": "Response is well-structured, professional, and clear.",
        "3": "Response has good structure but could be more professional or clearer.",
        "1": "Response lacks proper structure or professionalism."
    }
)

structure_professionalism_metric = PointwiseMetric(
    metric="structure_professionalism",
    metric_prompt_template=structure_professionalism_template
)

# Create evaluation dataset
eval_dataset = pd.DataFrame(ground_truth_data)

# Run evaluation
eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[structure_professionalism_metric],
    experiment="kyc-structure-professionalism"
)

eval_result = eval_task.evaluate()

# Display results
display_eval_result(eval_result)

# Create visualization of results
plot_bar_plot([("Structure and Professionalism", eval_result.summary_metrics, eval_result.metrics_table)])

The `input_variables` parameter is empty. Only the `response` column is used for computing this model-based metric.


Associating projects/305610648548/locations/us-central1/metadataStores/default/contexts/kyc-structure-professionalism-0bb6323c-9f98-4c72-bbe0-ad689269b95f to Experiment: kyc-structure-professionalism


Computing metrics with a total of 2 Vertex Gen AI Evaluation Service API requests.


100%|██████████| 2/2 [00:01<00:00,  1.19it/s]

All 2 metric requests are successfully computed.
Evaluation Took:1.6882728329983365 seconds


### Summary Metrics

,row_count,structure_professionalism/mean,structure_professionalism/std
0,2.0,5.0,0.0


### Row-based Metrics

,prompt,reference,response,structure_professionalism/explanation,structure_professionalism/score
0,\nYour task is to provide a comprehensive and ...,Here is a comprehensive report of negative new...,Here's a comprehensive report of negative news...,"The response is well-structured, professional ...",5.0
1,\nYour task is to provide a comprehensive and ...,Here is a comprehensive report of negative new...,Here's a comprehensive report of negative news...,"The response is well-structured, clear, and ma...",5.0


## Cleaning up
This notebook primarily makes API calls to Google's Gemini models and does not create persistent resources in your Google Cloud project (like VMs, storage buckets, etc.) beyond the API usage itself. Therefore, specific cleanup steps for created resources are generally not required after running this notebook.

If you want to disable the Vertex AI API used, you can do so from the Google Cloud Console, but this would affect any other services or notebooks relying on it.